In [9]:
# Initial imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from pathlib import Path

%matplotlib inline

In [10]:
# Load Data 
housing_df = pd.read_csv(Path("../data/housing_data_encoded.csv"))
housing_df

,Identifier,Bedroom,Price,Average_Market_Days,RegionName_Crosby,RegionName_Houston,RegionName_Pasadena,RegionName_Porter,State_TX,Metro_Houston-The Woodlands-Sugar Land,CountyName_Harris County,CountyName_Montgomery County
0,2,1,67653.0,133.0,0,1,0,0,1,1,1,0
1,2,2,89995.0,133.0,0,1,0,0,1,1,1,0
2,2,3,116641.0,133.0,0,1,0,0,1,1,1,0
3,2,4,167849.0,133.0,0,1,0,0,1,1,1,0
4,2,5,303323.0,133.0,0,1,0,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1215,223,1,79380.0,100.0,0,0,1,0,1,1,1,0
1216,223,2,121517.0,100.0,0,0,1,0,1,1,1,0
1217,223,3,159204.0,100.0,0,0,1,0,1,1,1,0
1218,1811,2,157660.0,108.0,0,0,0,1,1,1,0,1


In [3]:
housing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1220 entries, 0 to 1219
Data columns (total 13 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Date                                    1220 non-null   object 
 1   Identifier                              1220 non-null   int64  
 2   Bedroom                                 1220 non-null   int64  
 3   Price                                   1151 non-null   float64
 4   Average_Market_Days                     1030 non-null   float64
 5   RegionName_Crosby                       1220 non-null   int64  
 6   RegionName_Houston                      1220 non-null   int64  
 7   RegionName_Pasadena                     1220 non-null   int64  
 8   RegionName_Porter                       1220 non-null   int64  
 9   State_TX                                1220 non-null   int64  
 10  Metro_Houston-The Woodlands-Sugar Land  1220 non-null   int6

In [4]:
housing_df= housing_df.dropna()

In [5]:
# Creating the X and y sets
X = housing_df.drop(columns=["Price"]).values
y = housing_df["Price"].values

In [6]:
# Import the StandardScaler from sklearn
from sklearn.preprocessing import StandardScaler

# Scale the data
scaler = StandardScaler().fit(X)
X = scaler.transform(X)

ValueError: could not convert string to float: '2010-01-31'

In [24]:
# Creating training, validation, and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

X_train, X_val, y_train, y_val = train_test_split(X_test, y_test, random_state=0)

In [25]:
# Build and train the Deep Learning Model

# Import Keras modules for model creation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [26]:
# Model set-up
number_input_features = 11
hidden_nodes_layer1 = 15
hidden_nodes_layer2 = 10

In [27]:
# Define the LSTM RNN model
model = Sequential()

# Layer 1
model.add(
    Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Layer 2
model.add(Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
model.add(Dense(1, activation="swish"))

In [28]:
# Compile the model
model.compile(
    loss="mse",
    optimizer="adam",
    metrics=[
        "accuracy",
        tf.keras.metrics.TruePositives(name="tp"),
        tf.keras.metrics.TrueNegatives(name="tn"),
        tf.keras.metrics.FalsePositives(name="fp"),
        tf.keras.metrics.FalseNegatives(name="fn"),
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
        tf.keras.metrics.AUC(name="auc"),
    ],
)

In [29]:
# Summarize the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 15)                180       
_________________________________________________________________
dense_4 (Dense)              (None, 10)                160       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 11        
Total params: 351
Trainable params: 351
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Training the model
batch_size = 2000
epochs = 50
training_history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=epochs,
    batch_size=batch_size,
    verbose=1,
)

Epoch 1/50


InvalidArgumentError:  assertion failed: [predictions must be >= 0] [Condition x >= y did not hold element-wise:] [x (sequential_1/dense_5/IdentityN:0) = ] [[-0.159601226][0.240312919][-0.188596562]...] [y (Cast_6/x:0) = ] [0]
	 [[{{node assert_greater_equal/Assert/AssertGuard/else/_1/assert_greater_equal/Assert/AssertGuard/Assert}}]] [Op:__inference_train_function_4867]

Function call stack:
train_function
